<a href="https://colab.research.google.com/github/InowaR/colab/blob/main/simulate_collisions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [47]:
import torch
import gc
from tqdm import tqdm

def simulate_collisions():
    s, f, n = 10, 2, 50

    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    print(f"Используется: {device}")

    try:
        collisions = 0

        p = torch.randint(0, s, (n, 2), device=device)
        v = torch.randint(-1, 2, (n, 2), device=device)

        active_mask = torch.ones(n, dtype=torch.bool, device=device)

        for frame in tqdm(range(f), desc="Кадры"):
            with torch.no_grad():
                p[active_mask] += v[active_mask]

                out_of_bounds_mask = (p[:, 0] < 0) | (p[:, 0] >= s) | (p[:, 1] < 0) | (p[:, 1] >= s)

                active_mask = active_mask & ~out_of_bounds_mask

                if not active_mask.any():
                    print(f"Все частицы пропали на кадре {frame}")
                    break

                pos = p[active_mask]

                pos_clamped = pos.clamp(0, s-1)

                keys = pos_clamped[:, 0].long() * s + pos_clamped[:, 1].long()
                unique, counts = torch.unique(keys, return_counts=True)
                collisions += (counts * (counts - 1) // 2).sum().item()

        active_particles = active_mask.sum().item()
        print(f"\nЧастиц: {n:,}, Кадров: {f}, Коллизии: {collisions:,}")
        print(f"Осталось активных частиц: {active_particles:,}")

    except Exception as e:
        print(f"Ошибка: {e}")
    finally:
        gc.collect()
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
        print("Память очищена")

simulate_collisions()

Используется: cuda


Кадры: 100%|██████████| 2/2 [00:00<00:00, 659.64it/s]


Частиц: 50, Кадров: 2, Коллизии: 15
Осталось активных частиц: 31
Память очищена
